## Programmatic Interaction with Neuroglancer States

Visualizing data in Neuroglancer is one of the easiest ways to explore it in its full context.
The python package `nglui` was made to make it easy to generate Neuroglancer states from data, particularly pandas dataframes, in a progammatic manner.
The package can be installed with `pip install nglui`.

:::{important}
The `nglui` package interacts prominantly with `caveclient` and annotations queried from the database.
See the section on [querying the database](em:query-tables-section) to learn more.
:::

### Parsing Neuroglancer states

The `nglui.parser` package can be used to parse Neuroglancer states.  

The simplest way to parse the annotations in a Neuroglancer state is to first save the state using the Share button, and then copy the state id (the last number in the URL).  
For example, for the share URL `https://neuroglancer.neuvue.io/?json_url=https://global.daf-apis.com/nglstate/api/v1/5560000195854336`, the state id is `5560000195854336`  

You can then download the json and then use the `annotation_dataframe` function to generate a comprehensive dataframe of all the annotations in the state.  

In [ ]:
import os
from caveclient import CAVEclient
from nglui import parser

client = CAVEclient('minnie65_public')

state_id = 5560000195854336
state = client.state.get_state_json(state_id)
parser.annotation_dataframe(state)

Note that tags in the dataframe are stored as a list of integers, with each integer corresponding to one of the tags in the list.  
To get the mapping between the tag index and the tag name for each layer, you can use the `tag_dictionary` function.

In [ ]:
parser.tag_dictionary(state, layer_name='syns_out')

### Generating Neuroglancer States from Data

The `nglui.statebuilder` package is used to build Neuroglancer states that express arbitrary data.
The general pattern is that one makes a "StateBuilder" object that has rules for how to build a Neuroglancer state layer by layer, including selecting certain neurons, and populate layers of annotations.  
You then pass a DataFrame to the StateBuiler, and the rules tell it how to render the DataFrame into a Neuroglancer link.  
The same set of rules can be used on similar dataframes but with different data, such as synapses from different neurons.  
To understand the detailed use of the package, please see the [tutorial](https://github.com/seung-lab/NeuroglancerAnnotationUI/blob/master/examples/statebuilder_examples.ipynb).

However, a number of basic helper functions allow `nglui` to be used for common functions in just a few lines.

For example, to generate a Neuroglancer state that shows a neuron and its synaptic inputs and outputs, we can use the `make_neuron_neuroglancer_link` helper function.

In [ ]:
from nglui.statebuilder import helpers

helpers.make_neuron_neuroglancer_link(
    client,
    864691135122603047,
    show_inputs=True,
    show_outputs=True,
)

The main helper functions are:

* `make_neuron_neuroglancer_link` - Shows one or more neurons and, optionally, synaptic inputs and/or outputs.
* `make_synapse_neuroglancer_link` - Using a pre-downloaded synapse table, make a link that shows the synapse and the listed synaptic partners.
* `make_point_statebuilder` - Generate a statebuilder that to map a dataframe containing points (by default, formatted like a cell types table) to a Neuroglancer link.

In all cases, please look at the docstrings for more information on how to use the functions.